In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

df_user = pd.concat([train_user, test_user])

In [4]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

df_voc = pd.concat([train_voc, test_voc])

In [5]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

df_sms = pd.concat([train_sms, test_sms])

In [6]:
train_user.shape, test_user.shape

((6106, 6), (2045, 5))

In [7]:
train_voc.shape, test_voc.shape

((709144, 8), (276522, 8))

In [8]:
len(set(train_user['phone_no_m']) & set(train_voc['phone_no_m']))

4823

In [9]:
len(set(test_user['phone_no_m']) & set(test_voc['phone_no_m']))

1965

In [10]:
train_voc_calltype_id_1 = train_voc.loc[train_voc['calltype_id'] == 1, :].copy()

train_voc_calltype_id_2 = train_voc.loc[train_voc['calltype_id'] == 2, :].copy()

In [11]:
len(set(train_user['phone_no_m']) & set(train_voc_calltype_id_1['phone_no_m']))

4648

In [12]:
len(set(train_user['phone_no_m']) & set(train_voc_calltype_id_2['opposite_no_m']))

24

In [13]:
len(set(test_user['phone_no_m']) & set(test_voc['opposite_no_m']))

11

In [14]:
len(set(train_user['phone_no_m']) & set(train_sms['phone_no_m']))

4867

In [15]:
len(set(test_user['phone_no_m']) & set(test_sms['phone_no_m']))

2008

In [16]:
len(set(train_user['phone_no_m']) & set(train_sms['opposite_no_m']))

3

In [17]:
len(set(test_user['phone_no_m']) & set(test_sms['opposite_no_m']))

4

In [18]:
user_phone_no_m = df_user[['phone_no_m']].copy()

In [19]:
user_phone_no_m = user_phone_no_m.merge(df_voc, on='phone_no_m', how='left')
len(set(user_phone_no_m['phone_no_m']))

8151

In [22]:
len(set(user_phone_no_m[user_phone_no_m['start_datetime'].notnull()]['phone_no_m']))

6788